In [53]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
import pandas as pd 
from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.metrics import accuracy_score,precision_score, recall_score, f1_score
from sklearn.metrics import confusion_matrix,roc_auc_score,classification_report
from sklearn.model_selection import train_test_split
from numpy import hstack

import pickle 
import pandas as pd
import numpy as np

In [54]:
with open("./models/LabelEncoders_dic.pickle","rb") as f:
    encoder_dic=pickle.load(f)
with open("./models/MinMaxScalers_dic.pickle","rb") as f:
    scaler_dic=pickle.load(f)
    
with open("./models/combiner_model.pickle","rb") as f:
    combiner_model=pickle.load(f)
    
with open("./models/models_names.pickle","rb") as f:
    models_names=pickle.load(f)
    
models = []
for model in models_names:
    with open("./models/"+model+".pickle","rb") as f:
        models.append(pickle.load(f))

In [55]:
def transformer_df(df):
    for c in df.columns :
        if (df[c].dtype =="object"):
            encoder = encoder_dic[c]
            df[c]=encoder.transform(df[c])
    return df

In [56]:
def normaliser_all_columns(df):
    for c in df.columns :
        df[c]=scaler_dic[c].transform(df[c].values.reshape(-1,1))
    return df

In [57]:
# make predictions with stacked model
def super_learner_predictions(X, models, meta_model):
    meta_X = list()
    for model in models:
        yhat = model.predict_proba(X)
        meta_X.append(yhat)
    meta_X = hstack(meta_X)
    # predict
    return meta_model.predict(meta_X)

In [58]:
df =pd.read_csv("../../../data/full_data_small_datased.csv")

In [59]:
test_df = pd.DataFrame(columns=df.columns)
test_df = test_df.append(df.iloc[0])
test_df.head()

,MI_dir_L5_weight,MI_dir_L5_mean,MI_dir_L5_variance,MI_dir_L3_weight,MI_dir_L3_mean,MI_dir_L3_variance,MI_dir_L1_weight,MI_dir_L1_mean,MI_dir_L1_variance,MI_dir_L0.1_weight,...,HpHp_L0.1_covariance,HpHp_L0.1_pcc,HpHp_L0.01_weight,HpHp_L0.01_mean,HpHp_L0.01_std,HpHp_L0.01_magnitude,HpHp_L0.01_radius,HpHp_L0.01_covariance,HpHp_L0.01_pcc,Class
0,1.0,60.0,0.0,1.0,60.0,0.0,1.0,60.0,0.0,1.0,...,0.0,0.0,1.0,60.0,0.0,60.0,0.0,0.0,0.0,0.0


In [60]:
target = test_df.Class
x_test = test_df.drop(['Class'],axis=1)

In [61]:
x_test=transformer_df(x_test)
x_test=normaliser_all_columns(x_test)

In [62]:
# evaluate meta model
yhat = super_learner_predictions(x_test, models, combiner_model)
yhat

array([0.])